# Load Duns Data

Find and format File

In [3]:
filepath='../../1-Data/'
filename='dunswithlatlng.csv'
dunsdata=pd.read_csv(filepath+filename,sep='|',encoding='utf-8',
                    dtype={
                        'companyname':str,
                          'countrycode':str,
                          'countryname':str,
                           'street':str,
                           'city':str,
                          'postalcode':str,
                          'registerid':str,
                          'source_level':str,
                            'source_relation':str,
                          'isglobalroot':bool,
                            'state':str,
                        'lat':str,
                        'lng':str
                    })

C:\Downloaded\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


dunsdata=pd.read_excel('testexcel.xlsx')

In [5]:
dunsdata.shape

(238756, 15)

In [6]:
dunsdata.dropna(subset=['parentdunsid'],inplace=True)
dunsdata=dunsdata.loc[dunsdata['parentdunsid'].isin(dunsdata['dunsid'])]
dunsdata.fillna('notavailable',inplace=True)

In [7]:
import neatmartinet as nm
for c in ['dunsid','parentdunsid']:
    dunsdata[c]=dunsdata[c].apply(lambda r:nm.convert_int_to_str(n=r,fillwithzeroes=9))
print(nm.checkuniqueid(dunsdata['dunsid']))

True


In [8]:
neopath='../../Neo4j_db/duns.graphdb/import/'
dunsdata.to_csv(neopath+'dunsdata_commasep.csv',encoding='utf-8',sep=',',index=False)

# Load in neo4j

### Connect to graph

In [9]:
import py2neo
graph = py2neo.Graph(user='neo4j',password='admin')

#### Load Duns

In [10]:
query_uploadduns='''
USING PERIODIC COMMIT 10000
  LOAD CSV WITH HEADERS FROM 'file:///dunsdata_commasep.csv' AS row
    MERGE (c:DunsID{dunsid: row.dunsid})
    ON CREATE SET
        c.dunsid=row.dunsid,
        c.companyname=row.companyname,
        c.countrycode=row.countrycode,
        c.countryname=row.countryname,
        c.street=row.street,
        c.city=row.city,
        c.postalcode=row.postalcode,
        c.registerid=row.registerid,
        c.source_level=row.source_level,
        c.parentdunsid=row.parentdunsid,
        c.source_relation=row.source_relation,
        c.isglobalroot=row.isglobalroot,
        c.lat=row.lat,
        c.lng = row.lng,
        c.creationdate=timestamp(),
        c.lastseen=timestamp()
    ON MATCH SET
        c.lastseen=timestamp(),
        c.lat=row.lat,
        c.lng = row.lng;
'''
a=graph.run(query_uploadduns)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 715215, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


#### Create Relations

In [12]:
query_dunsischildof='''
MATCH (a:DunsID),(b:DunsID)
WHERE b.dunsid = a.parentdunsid and not a.parentdunsid=a.dunsid
MERGE (a)-[r:isChildOf]->(b)
WITH a,r
SET r.source_relation = a.source_relation
'''
a=graph.run(query_dunsischildof)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 67625, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


# TO DO : 
- Write update queries for country root
- Wrie RegisterId relation

# Close